In [1]:
import time
import random
import pathlib
from os.path import isfile
import copy
import sys

import numpy as np
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn

from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

from resnet_mask import *
from utils import *

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
## args
layers = 56
prune_type = 'structured'
prune_rate = 0.9
prune_imp = 'L2'
epochs = 300
batch_size = 128
lr = 0.2
momentum = 0.9
wd = 1e-4

cfgs = {
    '18':  (BasicBlock, [2, 2, 2, 2]),
    '34':  (BasicBlock, [3, 4, 6, 3]),
    '50':  (Bottleneck, [3, 4, 6, 3]),
    '101': (Bottleneck, [3, 4, 23, 3]),
    '152': (Bottleneck, [3, 8, 36, 3]),
}
cfgs_cifar = {
    '20':  [3, 3, 3],
    '32':  [5, 5, 5],
    '44':  [7, 7, 7],
    '56':  [9, 9, 9],
    '110': [18, 18, 18],
}

In [4]:
train_data_mean = (0.5, 0.5, 0.5)
train_data_std = (0.5, 0.5, 0.5)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=4)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=256, shuffle=False, num_workers=4)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
model = ResNet_CIFAR(BasicBlock, cfgs_cifar['56'], 10).to(device)
image_size = 32

In [6]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr,
                            momentum=momentum, weight_decay=wd) #nesterov=args.nesterov)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [7]:
##### main 함수 보고 train 짜기
best_acc1 = 0.0
for epoch in range(epochs):
    
    acc1_train_cor, acc5_train_cor = train(trainloader, epoch=epoch, model=model, 
                                       criterion=criterion, optimizer=optimizer, device=device)
    acc1_valid_cor, acc5_valid_cor = validate(testloader, epoch=epoch, model=model, criterion=criterion, device=device)
    
    acc1_train = round(acc1_train_cor.item(), 4)
    acc5_train = round(acc5_train_cor.item(), 4)
    acc1_valid = round(acc1_valid_cor.item(), 4)
    acc5_valid = round(acc5_valid_cor.item(), 4)

    # remember best Acc@1 and save checkpoint and summary csv file
#     summary = [epoch, acc1_train, acc5_train, acc1_valid, acc5_valid]

    is_best = acc1_valid > best_acc1
    best_acc1 = max(acc1_valid, best_acc1)
    if is_best:
        summary = [epoch, acc1_train, acc5_train, acc1_valid, acc5_valid]
#         save_model(arch_name, args.dataset, state, args.save)
#     save_summary(arch_name, args.dataset, args.save.split('.pth')[0], summary)

summary

train 0 ====> Acc@1 16.014 Acc@5 64.560
valid 0 ====> Acc@1 26.460 Acc@5 81.580
train 1 ====> Acc@1 31.296 Acc@5 85.164
valid 1 ====> Acc@1 31.030 Acc@5 85.480
train 2 ====> Acc@1 42.508 Acc@5 90.450
valid 2 ====> Acc@1 40.770 Acc@5 86.810
train 3 ====> Acc@1 54.646 Acc@5 94.780
valid 3 ====> Acc@1 53.450 Acc@5 94.650
train 4 ====> Acc@1 63.892 Acc@5 96.696
valid 4 ====> Acc@1 57.390 Acc@5 94.820
train 5 ====> Acc@1 69.438 Acc@5 97.636
valid 5 ====> Acc@1 66.830 Acc@5 97.260
train 6 ====> Acc@1 73.354 Acc@5 98.162
valid 6 ====> Acc@1 68.070 Acc@5 97.550
train 7 ====> Acc@1 76.302 Acc@5 98.550
valid 7 ====> Acc@1 73.530 Acc@5 98.540
train 8 ====> Acc@1 78.568 Acc@5 98.788
valid 8 ====> Acc@1 77.310 Acc@5 98.550
train 9 ====> Acc@1 80.464 Acc@5 99.052
valid 9 ====> Acc@1 74.110 Acc@5 97.460
train 10 ====> Acc@1 81.540 Acc@5 99.106
valid 10 ====> Acc@1 79.940 Acc@5 99.010
train 11 ====> Acc@1 83.090 Acc@5 99.264
valid 11 ====> Acc@1 75.810 Acc@5 98.640
train 12 ====> Acc@1 84.050 Acc@5 99

In [ ]:
# # print(len(trainloader))
# # epochs = 10
# # device = 'cpu'

# # epochs = 1
# for epoch in range(epochs):  # loop over the dataset multiple times

#     running_loss = 0.0
#     lr_sche.step()
#     for i, data in enumerate(trainloader, 0):
#         # get the inputs
#         inputs, labels = data
#         inputs = inputs #.to(device)
#         labels = labels #.to(device)

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 30 == 29:    # print every 30 mini-batches
#             # value_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(trainloader) ]))
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 30))
#             running_loss = 0.0
    
#     #Check Accuracy
# #     acc = acc_check(model, testloader, epoch, save=1)
#     # value_tracker(acc_plt, torch.Tensor([acc]), torch.Tensor([epoch]))
    

# print('Finished Training')